# Fake news detection using ML


In [1]:
# #About dataset
# id: unique id for a news article
# title: the title of a news article
# author: author of the news article
# text: the text of the article; could be incomplete
# label: a label that marks the article as potentially unreliable
#                1: fake news
#                0: real news

In [2]:
import numpy as np
import pandas as pd
import re #for searching words in text/para
from nltk.corpus import stopwords #to remove non imp words from dataset
from nltk.stem.porter import PorterStemmer #gives root word of a particular word
from sklearn.feature_extraction.text import TfidfVectorizer #converts text to feature
from sklearn.model_selection import train_test_split #splits data into trainig and test data
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Akanksha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
#printing stopwords
print(stopwords.words('English'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Data preprocessing

In [5]:
news_dataset = pd.read_csv(r'C:\Users\Akanksha\Downloads\fake news detection\train.csv')
news_dataset

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [6]:
news_dataset.shape

(20800, 5)

In [7]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
#counting number of missing values
news_dataset.isnull().sum()
#this counts no of missing values in each column

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
# since we have a large dataset, we can delete the columns containing
# null values or can even replace it with empty string
# since our dataset is very large, we gave enough data to train our
# model that's why we can do this

In [10]:
news_dataset = news_dataset.fillna('')
#replaced null values with empty string

In [11]:
# for our project, we are using title and author for training model.
# Since text is a long format, we will combine author and title and 
# train our dataset

In [12]:
#merging title and author column together
#creating a seperate column to store the result of concatenation
news_dataset['content'] = news_dataset['author']+''+news_dataset['title']

In [13]:
print(news_dataset['content'])

0        Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1        Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...
2        Consortiumnews.comWhy the Truth Might Get You ...
3        Jessica Purkiss15 Civilians Killed In Single U...
4        Howard PortnoyIranian woman jailed for fiction...
                               ...                        
20795    Jerome HudsonRapper T.I.: Trump a ’Poster Chil...
20796    Benjamin HoffmanN.F.L. Playoffs: Schedule, Mat...
20797    Michael J. de la Merced and Rachel AbramsMacy’...
20798    Alex AnsaryNATO, Russia To Hold Parallel Exerc...
20799               David SwansonWhat Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [14]:
#we will use this content column to train our model
#we will use this content data and labels to make predictions
#therefore seperating content column and label column

In [15]:
X = news_dataset.drop(columns='label', axis=1)
#if removing row-> axis=0
#if removing column->axis=1

Y = news_dataset['label']
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

### Stemming
<!-- it is the process of reducing a word to its root word
example - actor,actress,acting --> act -->

In [16]:
port_stem = PorterStemmer()
#loading PorterStemmer function to port_stem variable

In [17]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [18]:
#now applying stemming function to our content column

In [19]:
#editing the existing column by stemming
news_dataset['content'] = news_dataset['content'].apply(stemming)


In [20]:
#printing the stemmed content
print(news_dataset['content'])
#all the words will get transformed to their root word &
#all the insignificant words will be removed

0        darrel lucushous dem aid even see comey letter...
1        daniel j flynnflynn hillari clinton big woman ...
2                consortiumnew comwhi truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoyiranian woman jail fiction unpub...
                               ...                        
20795    jerom hudsonrapp trump poster child white supr...
20796    benjamin hoffmann f l playoff schedul matchup ...
20797    michael j de la merc rachel abramsmaci said re...
20798    alex ansarynato russia hold parallel exercis b...
20799                        david swansonwhat keep f aliv
Name: content, Length: 20800, dtype: object


In [21]:
#seperating the dala and label
X = news_dataset['content'].values
Y = news_dataset['label'].values
 

In [22]:
print(X)

['darrel lucushous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynnflynn hillari clinton big woman campu breitbart'
 'consortiumnew comwhi truth might get fire' ...
 'michael j de la merc rachel abramsmaci said receiv takeov approach hudson bay new york time'
 'alex ansarynato russia hold parallel exercis balkan'
 'david swansonwhat keep f aliv']


In [23]:
print(Y)
#this shows the label
#1 -> fake news
#0 -> original news

[1 0 1 ... 0 1 1]


In [24]:
Y.shape

(20800,)

In [ ]:
# X and Y are the values which arw going to be used for training the model
# X is still in textual format but 
# computer cannot understand this text so we need to convert it into
# meaningful numbers that the computer can understand 
# for this, we will use vectorization function.

In [25]:
#converting textual data to numerical data
vectorizer = TfidfVectorizer() #created a variable named vectorizer
#and stored vectorizer function in it

# tf - > term frequency
# idf -> inverse document frequency
# this basically counts how many times a particular word is being repeated 
# in a document. 
# Repeatation means it is an important word and hence it gets assigned by
# a particular number 


#fitting the vectorization function with X values
#all the words will get converted to their respective feature vectors
vectorizer.fit(X)
X = vectorizer.transform(X)


In [26]:
print(X)

  (0, 26340)	0.28088379401596425
  (0, 22724)	0.2552336018069161
  (0, 15019)	0.43006226759639316
  (0, 14555)	0.29177259684200296
  (0, 12782)	0.24619727512767195
  (0, 8022)	0.23133661742488731
  (0, 6273)	0.2839932825877813
  (0, 5969)	0.35488202138141456
  (0, 5006)	0.2472595823572816
  (0, 4211)	0.3625320323150658
  (0, 578)	0.2694167078545385
  (1, 27923)	0.36911845953845024
  (1, 11313)	0.24166773097712638
  (1, 8772)	0.5258635625386451
  (1, 5916)	0.31810058109638056
  (1, 4767)	0.23338756776626793
  (1, 3859)	0.45980466668763476
  (1, 3281)	0.18652439327549428
  (1, 2622)	0.3562953366945267
  (2, 26235)	0.3665032495181434
  (2, 16361)	0.43295215406038445
  (2, 9454)	0.30743020569262086
  (2, 8567)	0.3411947414020896
  (2, 5240)	0.40440534260277944
  (2, 5121)	0.5511414848555652
  :	:
  (20797, 25776)	0.08220218573989037
  (20797, 25319)	0.3119640221826561
  (20797, 22086)	0.24902354987792552
  (20797, 20778)	0.2729578683228216
  (20797, 20493)	0.249994989010826
  (20797, 17505

<!-- Now textual data has been converted to numeric data
so feeding this data to our dataset -->

# Splitting the dataset to training and test data

In [ ]:
#0.2 means 20% data goes into training the model
#using stratify means real and fake news will be segregated in equal proportion

In [31]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2, stratify=Y, random_state=2)

# Training the logistic regression model

In [34]:
model = LogisticRegression()

In [35]:
#training model 
model.fit(X_train , Y_train)

LogisticRegression()

# Evalutating the model


In [38]:
#checking accuracy store on the training data
#all the values predicted by model will be stored in this variable
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
#X_train_prediction this is the values predicted and Y_train is the 
#original value

In [43]:
print('Accuracy score of the trainig data: ', training_data_accuracy)

Accuracy score of the trainig data:  0.9719951923076923


In [40]:
#Finding accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [45]:
print('Accuracy score of the test data: ', test_data_accuracy)

Accuracy score of the test data:  0.9548076923076924


# Building a predictive system


In [47]:
X_new = X_test[0]
prediction = model.predict(X_new)
print(prediction)

if(prediction[0] == 0):
    print('News is real')
else:
    print('The news is fake')

[1]
The news is fake


In [48]:
#checking if the predicted result is true or not
print(Y_test[0])

1


In [49]:
X_new = X_test[1]
prediction = model.predict(X_new)
print(prediction)

if(prediction[0] == 0):
    print('News is real')
else:
    print('The news is fake')

[0]
News is real


In [50]:
print(Y_test[1])

0
